In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import statsmodels.api as sm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv


In [2]:
#Importing data

df_train = pd.read_csv ('/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv')
df_calendar = pd.read_csv ('/kaggle/input/m5-forecasting-accuracy/calendar.csv')
df_sell_prices = pd.read_csv ('/kaggle/input/m5-forecasting-accuracy/sell_prices.csv')

In [3]:
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

df_train = reduce_mem_usage(df_train)
df_calendar = reduce_mem_usage(df_calendar)
df_sell_prices = reduce_mem_usage(df_sell_prices)

Mem. usage decreased to 95.00 Mb (78.7% reduction)
Mem. usage decreased to  0.12 Mb (41.9% reduction)
Mem. usage decreased to 130.48 Mb (37.5% reduction)


In [4]:
df_train.head()
#df_calendar.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4


Converting the data format from wide to long format

In [5]:
sales_data = df_train.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                                         var_name='d',
                                         value_name='sales')


In [6]:
nrow = df_train.shape[0]
nrow

30490

In [7]:
import time
start = time.process_time()
# your code here    
print(time.process_time() - start)

4.684799998244671e-05


In [8]:
#Executing the model for all combinations

def my_function(i):
    
        start = time.process_time()
        id = df_train.at[i,'id']
        #print(time.process_time() - start)
        #start = time.process_time()
        sales_data_test = df_train[(df_train['id']==id)]
        #print(time.process_time() - start)
        #start = time.process_time()
        sales_data_test = sales_data_test.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'],
                                             var_name='d',
                                             value_name='sales')
        #print(time.process_time() - start)
        data  = pd.merge(sales_data_test,df_calendar,how='inner',on='d')
        data = data.set_index(['date'])


        # define model configuration
        my_order = (1, 1, 1)
        my_seasonal_order = (1, 1, 1, 12)
        # define model
        model = sm.tsa.statespace.SARIMAX(data['sales'], order=my_order, seasonal_order=my_seasonal_order)
        model_fit = model.fit()
        yhat = model_fit.predict(start='25-04-16', end='19-06-16')
        yhat = yhat.to_frame()
        yhat['id'] = id
        yhat['Date'] = yhat.index
        yhat = yhat.rename(columns={0: "Forecast_Sales"})
        return yhat

In [9]:

import multiprocessing as mp
from datetime import datetime
from joblib import Parallel, delayed
from tqdm import tqdm

mylist = list(range(0,5))
num_cores = mp.cpu_count()
start = time.time()
if __name__ == "__main__":
    processed_list = Parallel(n_jobs=num_cores)(delayed(my_function)(i) 
                                                     for i in mylist)

forecast_final = pd.DataFrame(columns=['id','Date','Forecast_Sales'])

for i in mylist :
    forecast_final = forecast_final.append(processed_list[i])
    
n = forecast_final.shape[0]
forecast_final.index = range(n)
    
for i, row in forecast_final.iterrows():
    if (forecast_final.loc[i,'Date'].date()) > datetime.strptime('2016-05-22', '%Y-%m-%d').date():
        forecast_final.loc[i,'id'] = forecast_final.loc[i,'id'] .replace('validation', 'evaluation')

print(time.time() - start)

30.487004041671753


In [10]:
forecast_final['d'] = forecast_final.sort_values(['id','Date'], ascending=[True,True]) \
             .groupby(['id']) \
             .cumcount() + 1
forecast_final['d'] = forecast_final['d'].astype(str).apply(lambda x: x.zfill(2))
forecast_final['d'] = 'F' + forecast_final['d'].astype(str)
forecast_final = forecast_final.drop(['Date'], axis = 1) 
print(forecast_final.head())

#df1 = df.set_index(['id', 'num', 'q'])['v'].unstack().reset_index()

#forecast_fin = forecast_final.set_index(['id', 'Date']),['d'],value_name='Forecast_Sales')



                              id  Forecast_Sales    d
0  HOBBIES_1_001_CA_1_validation        1.049144  F01
1  HOBBIES_1_001_CA_1_validation        1.091024  F02
2  HOBBIES_1_001_CA_1_validation        1.153179  F03
3  HOBBIES_1_001_CA_1_validation        0.958104  F04
4  HOBBIES_1_001_CA_1_validation        0.991806  F05


In [11]:

forecast_fin = pd.pivot(forecast_final, index = 'id', columns = 'd', values = 'Forecast_Sales').reset_index()
forecast_fin= forecast_fin.sort_values(['id'])
for i, row in forecast_fin.iterrows():
    if (forecast_fin.loc[i,'id'][-10:] ==  'validation') :
        forecast_fin.loc[i,'sort'] = 1;
    else :
        forecast_fin.loc[i,'sort'] = 2;
forecast_fin = forecast_fin.sort_values(['sort','id']).drop(['sort'],axis=1)

col_list = list(forecast_fin.columns)

for i in range(len(col_list)):
    if (col_list[i][1] == '0' ):
        col_list[i] = col_list[i].replace("0","")
forecast_fin.columns = col_list


**Creating the output file**

In [12]:
forecast_fin.to_csv('Sanjay_Gopal_Submission.csv',index=False)

**Code snippet for correlation analysis**

Checking correlation of items against events